The idea of functions:

 -Scale the word_counts and language_words with_frequency by the highest frequent word

In [1]:
import json

N = 100
def lang_confidence_score(word_counts, language_words_with_frequency):
    # zakładam że language_words_with_frequency to słownik
    top_N_words = language_words_with_frequency[:N]
    score = 0
    for word , freq in top_N_words.items():
        count_in_text = word_counts.get(word)
        score += count_in_text * freq
    sum = 0
    for word , count in word_counts.items():
        sum += count
    return score/sum

I choose english (the language which was used for writing pokemon wiki) and french


In [4]:
import wordfreq
ListOfWordsInEnglish = wordfreq.top_n_list(n = N , lang= 'en')
ListOfWordsInFrench = wordfreq.top_n_list(n = N , lang='fr')
ListOfWordsInPolish = wordfreq.top_n_list(n = N , lang= 'pl')
SetOfWordsInEnglish = {}
SetOfWordsInFrench = {}
SetOfWordsInPolish = {}
for i in range(N):
    SetOfWordsInEnglish[ListOfWordsInEnglish[i]] = (
                wordfreq.word_frequency(ListOfWordsInEnglish[i], lang='en')
            )
    SetOfWordsInFrench[ListOfWordsInFrench[i]] = (
                wordfreq.word_frequency(ListOfWordsInFrench[i], lang='fr')
            )
    SetOfWordsInPolish[ListOfWordsInPolish[i]] = (
                wordfreq.word_frequency(ListOfWordsInPolish[i], lang='pl')
            )

In [10]:
from wiki_scraper import Scraper
import json
obiekt = Scraper(use_local_html_file_instead=True)
if obiekt.count_words('Team Rocket') is None:
    print("Nie powiódł się count-words")
    raise Exception
with open("./word-count.json" , 'r' , encoding='utf-8') as file:
    word_count = json.load(file)



If you were looking for the TCG set, see Team Rocket (TCG) . For the group consisting of Jessie , James , and Meowth , see Team Rocket trio . Rocket redirects here. For the Trainer class referred to as Rocket in Generation I, see Team Rocket Grunts . Team Rocket ロケット 団 Rocket-dan Artwork of Team Rocket Grunts from HeartGold and SoulSilver Leader Giovanni Madame Boss (former; The Birth of Mewtwo ) Archer HG SS Masked Man (former; Adventures ) Carr (former; Adventures) Region Kanto , Johto , Sevii Islands (cross-canon) Sinnoh , Unova , and Alola ( Pokémon the Series ) Admins Archer , Ariana , Petrel , Proton ( core series ) Matori , Iron-Masked Marauder ( Pokémon the Series ) Lt. Surge , Koga , Sabrina , Harry, Al, Ken , Carl , Sham , Carr , Orm , Sird ( Adventures ) Major targets Mt. Moon , Pokémon Tower , Silph Co. , Mt. Ember , Icefall Cave , Dotted Hole , Five Island , Slowpoke Well , Lake of Rage , Goldenrod Radio Tower ( core series ) Ash Ketchum , Ash's Pikachu , Dreamyard , Musha